In [1]:
from cosecha_colectiva import miscelaneous as m
m.try_find_conf_file()


from cosecha_colectiva import main_tester as mt
from cosecha_colectiva import config
from cosecha_colectiva import query_cosecha as qc
from cosecha_colectiva import test_cosecha_api as tca

import logging
import pathlib
from datetime import datetime

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


[2023-05-21 18:41:02,937][INFO]: Connecting b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306
[2023-05-21 18:41:04,055][INFO]: Connected b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306


In [2]:
xls_name = 'CAF_GrupoMesero.xlsx'
xls_name = 'CAF_GrupoHidalgo.xlsx'
xls_name = 'GrupoCafam.xlsx'

In [3]:
id_grupo = qc.get_grupo_id_by_name(xls_name.split('.')[0])
print(id_grupo)
#qc.delete_grupo(id_grupo, solo_sesiones=True)
#qc.delete_grupo(id_grupo)

1734


In [4]:
qc.delete_grupo(id_grupo, force_delete=True)

([{'Transaccion_prestamo_id': 5124}, {'Transaccion_prestamo_id': 5134}, {'Transaccion_prestamo_id': 5144}, {'Transaccion_prestamo_id': 5154}, {'Transaccion_prestamo_id': 5164}, {'Transaccion_prestamo_id': 5174}, {'Transaccion_prestamo_id': 5184}], 'transaccion_prestamos')
DELETE from heroku_59f792929604e63.transaccion_prestamos WHERE Transaccion_prestamo_id in (5124, 5134, 5144, 5154, 5164, 5174, 5184)
([{'Transaccion_id': 21374}, {'Transaccion_id': 21384}, {'Transaccion_id': 21394}, {'Transaccion_id': 21404}, {'Transaccion_id': 21414}, {'Transaccion_id': 21424}, {'Transaccion_id': 21434}, {'Transaccion_id': 21444}, {'Transaccion_id': 21454}, {'Transaccion_id': 21464}, {'Transaccion_id': 21474}, {'Transaccion_id': 22304}, {'Transaccion_id': 22314}, {'Transaccion_id': 22324}, {'Transaccion_id': 22334}, {'Transaccion_id': 22344}, {'Transaccion_id': 22354}, {'Transaccion_id': 22364}, {'Transaccion_id': 22374}, {'Transaccion_id': 22384}, {'Transaccion_id': 22394}, {'Transaccion_id': 22404}

In [5]:
now = datetime.now()
log_date = now.strftime("%Y-%m-%d_%H_%M_%S")
log_file = pathlib.Path(config.log_dir, log_date+'CreacionGrupo.log')
logging.basicConfig(filename=log_file, encoding='utf-8', level=logging.INFO)

In [6]:
mt.test_crear_grupo(xls_name)

[2023-05-21 18:42:24,738][WARNING]: MySQL server has gone away. Reconnecting to the server.
[2023-05-21 18:44:16,168][WARNING]: MySQL server has gone away. Reconnecting to the server.


In [8]:
import time
admin_header,_ = tca.CAF_API_general.login_first_user_excel(xls_name)

tca.CAF_API_sessions_tester.create_session(id_grupo, admin_header)

id_sesion = qc.get_active_sesion(id_grupo)

time.sleep(0.1)
tca.CAF_API_group_creation_tester.create_acuerdos(xls_name)
time.sleep(0.1)
tca.CAF_API_group_creation_tester.acciones_iniciales(xls_name, id_grupo, id_sesion)
time.sleep(0.1)
tca.CAF_API_sessions_tester.end_session(id_grupo, admin_header)